create empty mne-python raw object

In [1]:
import numpy as np
import mne

# Create a simple RawArray
sfreq = 250  # Sampling frequency
ch_names = [f'EEG{d}' for d in range(1,75)]
ch_types = ["eeg"] * 74
info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types=ch_types)

data = np.random.randn(74, 747750)  # 2 channels, 1000 samples
raw = mne.io.RawArray(data, info)

print(isinstance(raw, mne.io.Raw))  # True
print(type(raw))  # <class 'mne.io.array.array.RawArray'>

Creating RawArray with float64 data, n_channels=74, n_times=747750
    Range : 0 ... 747749 =      0.000 ...  2990.996 secs
Ready.
False
<class 'mne.io.array.array.RawArray'>


braindecode call __getitem__ of mne.base.Raw, which then calls _getitem which calls _read_segment of BaseRaw. mne uses _read_segment to read a specific range of the file. We want to test whether S3 file via fsspec can be integrated
It calls _read_segments_file of the BaseRaw class. Any subclass must implement this method. EEGLAB calls fiff reader function: mne/_fiff/utils.py#L200

In [1]:
# To be able to make edits to repo without having to restart notebook
%load_ext autoreload
%autoreload 2
import sys 
sys.path.append('..')
from eegdash.data_utils import RawEEGDash

In [20]:
eegdash = RawEEGDash('s3://testspeedeegdash/sub-002_task-FaceRecognition_eeg.set', {'sfreq': 250, 'nchans': 74, 'n_times': 747750}, preload=True)

n_times 747750
Reading 0 ... 747749  =      0.000 ...  2990.996 secs...


In [23]:
eegdash.get_data()

array([[5.56413960e+07, 4.40817108e-39, 5.60519386e-45, ...,
        7.10578003e+01, 7.32979889e+01, 6.95856934e+01],
       [5.97010569e-07, 8.72444220e-39, 0.00000000e+00, ...,
        4.59728317e+01, 4.81444893e+01, 4.25833282e+01],
       [1.67969504e+08, 8.90820568e-39, 7.00649232e-45, ...,
        4.93649330e+01, 4.92341499e+01, 4.53561974e+01],
       ...,
       [1.00893489e-43, 7.84727140e-44, 8.90820568e-39, ...,
        4.53600616e+01, 4.10236855e+01, 4.29333000e+01],
       [1.07449142e-38, 8.40779079e-45, 1.02856414e-38, ...,
        5.49960251e+01, 4.60316620e+01, 4.72489014e+01],
       [4.13273465e-39, 1.12103877e-44, 1.01938998e-38, ...,
        4.54888268e+01, 3.74752045e+01, 2.96322441e+01]],
      shape=(74, 747750))

integrate with braindecode

In [33]:
from braindecode.datasets import BaseDataset, BaseConcatDataset
eegdash_braindecode = BaseConcatDataset([BaseDataset(eegdash)])

In [34]:
from braindecode.preprocessing import (
    preprocess, Preprocessor, create_fixed_length_windows)
windows_ds = create_fixed_length_windows(eegdash_braindecode, start_offset_samples=0, stop_offset_samples=None,
        window_size_samples=1000,
        window_stride_samples=1000, drop_last_window=True,
        preload=False)

In [36]:
windows_ds[0][0].shape

(74, 1000)